In [2]:
import os
from ultralytics import YOLO
from sklearn.metrics import classification_report
import cv2

# パスの設定 (適宜変更してください)
MODEL_PATH = "./runs/classify/result3/weights/best.pt"
TEST_DATA_ROOT = "./data/test"
SAVE_DIR = "./runs/classify/result3/predict"  # 保存ディレクトリのパス

# 保存ディレクトリを作成
os.makedirs(os.path.join(SAVE_DIR, "studio"), exist_ok=True)
os.makedirs(os.path.join(SAVE_DIR, "others"), exist_ok=True)


# モデルのロード
model = YOLO(MODEL_PATH)

true_labels = []
pred_labels = []

# テストデータのディレクトリ
test_dirs = [os.path.join(TEST_DATA_ROOT, d) for d in os.listdir(TEST_DATA_ROOT) if os.path.isdir(os.path.join(TEST_DATA_ROOT, d))]

for test_dir in test_dirs:
    label = os.path.basename(test_dir)  # ディレクトリ名から正解ラベルを取得
    image_files = [os.path.join(test_dir, f) for f in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, f))]

    for image_file in image_files:
        results = model.predict(source=image_file, save=False, imgsz=640)
        if results[0].probs is not None:
            predicted_class = results[0].probs.top1
            predicted_label = model.names[predicted_class]

            true_labels.append(label)
            pred_labels.append(predicted_label)

            # 画像を読み込み
            img = cv2.imread(image_file)

            # 保存先のパスを決定
            save_path = os.path.join(SAVE_DIR, predicted_label, os.path.basename(image_file))

            # 画像を保存
            cv2.imwrite(save_path, img)


# 分類レポートの出力
print(classification_report(true_labels, pred_labels))

# 各クラスごとの精度、再現率、F1値、サポートも表示 (より詳細な情報)
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1, support = precision_recall_fscore_support(true_labels, pred_labels)

for i, label in enumerate(model.names): # model.names でラベル名を取得
    print(f"Label: {label}")
    print(f"  Precision: {precision[i]}")
    print(f"  Recall: {recall[i]}")
    print(f"  F1-score: {f1[i]}")
    print(f"  Support: {support[i]}")


image 1/1 /home/omichi/segment-anything/codes_yolo/data/test/anomaly/56.jpg: 640x640 anomaly 1.00, normal 0.00, 3.2ms
Speed: 19.1ms preprocess, 3.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/omichi/segment-anything/codes_yolo/data/test/anomaly/4.jpg: 640x640 anomaly 1.00, normal 0.00, 3.2ms
Speed: 15.3ms preprocess, 3.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/omichi/segment-anything/codes_yolo/data/test/anomaly/n3_71.jpg: 640x640 anomaly 0.99, normal 0.01, 3.2ms
Speed: 15.4ms preprocess, 3.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/omichi/segment-anything/codes_yolo/data/test/anomaly/18.jpg: 640x640 anomaly 0.87, normal 0.13, 3.2ms
Speed: 15.9ms preprocess, 3.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/omichi/segment-anything/codes_yolo/data/test/anomaly/n3_47.jpg: 640x640 anomaly 0.94, normal 0.06, 3.2ms
Speed: 15.0m

: 